In [1]:
import numpy as np
import pandas as pd
import glob
import os.path as op
from os import makedirs
import shutil

def convert_parfile(infile, outfile, duration=18):
    parfile = pd.read_table(infile, header=None, names=['onset','blocktype','trial_type','color'])
    assert(len(np.unique(parfile.blocktype))==3) # we only expect M, P, and blank blocks.
    parfile['duration'] = duration # default 18 (3T), 16 for 7T
    out = parfile[['onset','duration','trial_type']]
    out.to_csv(outfile,"\t", index=False)

def convert_parfiles(in_glob, out_stem):
    parfiles = glob.glob(in_glob)
    tsvfiles = [out_stem + op.basename(p).split('run')[-1].split('.')[0] + '_events.tsv' for p in parfiles]
    for (par, tsv) in zip(parfiles, tsvfiles):
        convert_parfile(par, tsv)
    

In [ ]:
#convert 20150401
convert_parfiles("/Users/smerdis/data/LGN/MS_LGN/20150401MS.dicom/20150401MS/Stimuli/parfiles/MS_20150401_run*.par",
                 "/Users/smerdis/data/LGN/BIDS/AlexLGN_no0327/sub-MS/ses-20150401/func/sub-MS_ses-20150401_task-mp_run-")

In [2]:
convert_parfiles("/Users/smerdis/data/LGN/MS_LGN/20150519MS.dicom/20150519MS/Stimuli/parfiles/MS_20150519_run*.par",
                 "/Users/smerdis/data/LGN/BIDS/AlexLGN_no0327/sub-MS/ses-20150519/func/sub-MS_ses-20150519_task-mp_run-")

In [ ]:
def migrate_alex2015():
    sessions_to_group = {#'20150327':'/Users/smerdis/data/LGN/MS_LGN/20150327MS.dicom/20150327MS/20150327MS_dicom/',
    '20150401':'/Users/smerdis/data/LGN/MS_LGN/20150401MS.dicom/20150401MS/20150401MS_backup/',
    '20150519':'/Users/smerdis/data/LGN/MS_LGN/20150519MS.dicom/20150519MS/20150519MS_dicom/'}

    dataset_name = 'AlexLGN_no0327'
    sub_code = 'MS'
    staging_folder = op.join('/Users/smerdis/data/LGN', dataset_name, sub_code)
    if not op.exists(staging_folder):
        makedirs(staging_folder)
        
    print(sessions_to_group)
    
    for k,v in sessions_to_group.items():
        session_folder = op.join(staging_folder, k)
        if not op.exists(session_folder):
            makedirs(session_folder)
        dicoms = glob.glob((v + "epi*/IM-*.dcm"))
        for f in dicoms:
            shutil.copy(f, session_folder) #put all dicoms from that day in one folder


In [ ]:
migrate_alex2015()

In [ ]:
['L', 'R'] * 6

In [34]:
def write_hemifield_localizer_event_file(event_file):
    num_LR_cycles = 11
    len_hemifield_cycle = 13.5
    TR = 2.25

    #total_stim_length = (len_hemifield_cycle * 2) * num_LR_cycles
    #total_volumes = total_stim_length/TR
    volumes_per_hemicycle = len_hemifield_cycle/TR

    conds = ['R', 'L'] # we start with L, actually
    #print(total_stim_length, volumes_per_hemicycle, total_volumes)

    file_contents = "onset\tduration\ttrial_type\n"
    for hemicycle in range(1, num_LR_cycles*2+1):
        file_contents += f"{hemicycle*len_hemifield_cycle}\t{len_hemifield_cycle}\t{conds[hemicycle%len(conds)]}\n"

    with open(event_file, 'w') as f:
        f.write(file_contents)

In [35]:
from bids.grabbids import BIDSLayout
raw_layout = BIDSLayout("/Users/smerdis/data/LGN/BIDS/AlexLGN_no0327/")
hemifield_event_files = raw_layout.get(subject='MS', modality='func', 
                                       task="hemi", extensions=['.tsv'])
for f in hemifield_event_files:
    write_hemifield_localizer_event_file(f.filename)

In [29]:
list(range(1, num_LR_cycles*2+1))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [14]:
%ls

Event file conversion.ipynb
LGN_MP_Connectivity_Analyses.xlsx
LGN_MP_Functional_Connectivity_20140407.key
LGN_MP_Spatial_Analyses2_20140407.key
LGN_MP_Spatial_Analyses_20140407.key
Pipeline_AS/
Porcupine/
Untitled.ipynb
Untitled1.ipynb
build-Porcupine-Desktop_Qt_5_10_0_clang_64bit-Release/
build-Porcupine-Desktop_Qt_5_9_3_clang_64bit-Release/
figures/
glm2/
glm_pipeline/
megavista/
pycortex-gallantlab/
